In [1]:
# First Installing the Optuna
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 9.3 MB/s eta 0:00:00


In [2]:
# Import Necessary libraries
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
# Loading the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Loading the dataset with specific columns we need only
df = pd.read_csv(url, names=columns)

# Checking the DataSet using its Header
df.head()

# NOTE OUR DATASET IS OF SHAPE (768, 9)
# MEANS WE HAVE 768 -> SAMPLES AND
# 9 -> FEATURES

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
print(f"Zero Initialized Values in Glucose Column: {(df['Glucose']==0).sum()}")
print(f"Zero Initialized Values in BloodPressure Column: {(df['BloodPressure']==0).sum()}")
print(f"Zero Initialized Values in SkinThickness Column: {(df['SkinThickness']==0).sum()}")
print(f"Zero Initialized Values in Insulin Column: {(df['Insulin']==0).sum()}")
print(f"Zero Initialized Values in BMI Column: {(df['BMI']==0).sum()}")
print(f"Zero Initialized Values in DiabetesPedigreeFunction Column: {(df['DiabetesPedigreeFunction']==0).sum()}")
print(f"Zero Initialized Values in Age Column: {(df['Age']==0).sum()}")

Zero Initialized Values in Glucose Column: 5
Zero Initialized Values in BloodPressure Column: 35
Zero Initialized Values in SkinThickness Column: 227
Zero Initialized Values in Insulin Column: 374
Zero Initialized Values in BMI Column: 11
Zero Initialized Values in DiabetesPedigreeFunction Column: 0
Zero Initialized Values in Age Column: 0


In [4]:
# From the Above Results we see that this dataset will definitely need some
# preprocessing like handling of missing values (initialized as '0' as seen
# in Header and the calculation made above)
# We need to RESET Custom Values Against the zeros in Glucose, BloodPressure
#   SkinThickness, Insulin and BMI columns. All other columns are ok.
import numpy as np

# Defining which columns need to be manipulated
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Replacing the "0" in respective columns with NAN so that we could manipulate
#   these columns easily with .fillna() function
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0,np.nan)

# Imputing the Values with the mean of respective columns
# df.means() -> returns array AND IN THIS PARTICULAR EXAMPLE IT WILL RETURN
#               ARRAY OF SHAPE (9,) MEANS 9 ROWS AND 1 COLUMN
print(df.mean().shape)
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

(9,)
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [5]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional Step: Scaling the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Checking the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Training label shape: {y_train.shape}')
print(f'Test set shape: {X_test.shape}')
print(f'Test label shape: {y_test.shape}')

# Now we have the following data configuration till now:
# No_of_Features in this example are '8'
# X_train (Training Feature) Matrix Shape (n_samples, n_features) -> (537, 8) In this Example
#     means we have 537 samples for training
# X_test (Testing Feature) Matrix Shape (n_samples, n_features) -> (231, 8) In this Example
#     means we have 231 samples for training

Training set shape: (537, 8)
Training label shape: (537,)
Test set shape: (231, 8)
Test label shape: (231,)


In [6]:
# Importing the Random Forest Classifier and Cross Validation Socrer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [7]:
# Now we will start defining our objective function
# objective function takes a parameter trial which is operated by optuna and
#   it gives everything we need from optuna like the next hyperparameter suggestion

# The objective function have the following flows and definitions inside:
# 1. Suggestion of Hyperparameters from trial object
# 2. Hyperparameters, DataLoaders and Model Initialization based on which Hyperparameters
#     are being tuned means if we are tuning learning rate then It must be defined
#     within the objective function and use the suggested value, and so for the
#     others like batch_size of dataloaders, n_estimators of Model etc.
# 3. Training Loop or mechanism which give either accuracy or loss or any other
#     metric which could be used as base for optimizing the complete hyperparameters
#     like if we want to increase the accuracy then accuracy must be extracted
#     from the training loop and MUST BE RETURNED

# We are going to tune the following Hyperparameters of RandomForest with respective search space
# n_estimators: Search Space = Range from 50 - 200 as integers (means 51, 53 etc.. Not 50.35)
# max_depth: Search Space = Range from 3 - 20 as integers (means 4,6 etc.. Not 3.5)

def objective(trial):
  # Suggesting the n_estimaters hyperparameter value
  n_estimaters = trial.suggest_int('n_estimators', 50, 200)
  # Suggesting the max_depth hyperparameter value
  max_depth = trial.suggest_int('max_depth', 3, 20)

  # Creating the Model using the suggested values
  model = RandomForestClassifier(
      n_estimators=n_estimaters,
      max_depth=max_depth,
      random_state=42
  )

  # Now peforming the training and getting the BASE METRIC for optuna
  # Calculating the accuracy score using Cross Validation Scorer with 3 Times
  #   Cross Validation and then taking the average accuracy score which is returned
  acc_score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

  # Returning the Accuracy Score because WE WANT OPTUNA TO MAXIMIZE THIS
  return acc_score

In [8]:
# Now Creating a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)  # We will be going to run 10 trails because of limitation of my hardware

[I 2026-02-05 19:18:31,998] A new study created in memory with name: no-name-c52a942c-180b-4e63-8dfe-c03d2020102a
[I 2026-02-05 19:18:32,888] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 147, 'max_depth': 5}. Best is trial 0 with value: 0.7672253258845437.
[I 2026-02-05 19:18:33,847] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 151, 'max_depth': 16}. Best is trial 1 with value: 0.7746741154562384.
[I 2026-02-05 19:18:35,528] Trial 2 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 84, 'max_depth': 6}. Best is trial 1 with value: 0.7746741154562384.
[I 2026-02-05 19:18:38,019] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 173, 'max_depth': 14}. Best is trial 1 with value: 0.7746741154562384.
[I 2026-02-05 19:18:39,210] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 109, 'max_depth': 14}. Best is trial 1 with value: 0.7746741

In [9]:
# Printing the Best Accuracy and Hyperparameters
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_params}")

Best trial accuracy: 0.7783985102420857
Best hyperparameters: {'n_estimators': 55, 'max_depth': 17}


In [10]:
# Extracting the Best Parameters dictionary
params = study.best_params
params

{'n_estimators': 55, 'max_depth': 17}

In [11]:
from sklearn.metrics import accuracy_score

# Now Training a New Model using the best hyperparameters that were gained by tuning
best_model = RandomForestClassifier(**params, random_state=42)

# Fit the model to the training data (In other words training the model)
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna
- Trying Out Different Samplers for optuna study
- For this we will be using the same imports and objective function
- We only change the sampler for optuna

In [12]:
# All the important imports are created and objective function is already defined

# Now Trying the RandomSampler as below
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=50)

[I 2026-02-05 19:18:57,056] A new study created in memory with name: no-name-b3c93332-b51d-4aa5-bfa0-d07aca26e96e
[I 2026-02-05 19:18:57,417] Trial 0 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 70, 'max_depth': 4}. Best is trial 0 with value: 0.7616387337057727.
[I 2026-02-05 19:18:57,790] Trial 1 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 71, 'max_depth': 17}. Best is trial 1 with value: 0.7802607076350093.
[I 2026-02-05 19:18:58,311] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 95, 'max_depth': 19}. Best is trial 1 with value: 0.7802607076350093.
[I 2026-02-05 19:18:59,096] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 129, 'max_depth': 11}. Best is trial 1 with value: 0.7802607076350093.
[I 2026-02-05 19:18:59,806] Trial 4 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 98, 'max_depth': 11}. Best is trial 1 with value: 0.780260707

In [13]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_params}')

Best trial accuracy: 0.7839851024208566
Best hyperparameters: {'n_estimators': 73, 'max_depth': 18}


In [14]:
# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


In [15]:
# Now we will be using GridSampler for optuna
# GridSampler tries out all the combination of samples we define in the
#   search space so we need to define a search space object to pass it to
#   Sampler for reference of values to be searched on
#   Note: the search space object must have keys with the same name as defined
#     in the objective function trial suggestion like
#     trial.suggest_int('n_estimators', 50, 200) -> so search space must contain
#     'n_estimators' key to define search space for n_estimators
#     If the key strings got mismatched either error or other unsuccessful
#     behaviours occur.
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [16]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
# A GridSampler will iterate over all the combination of samples so n_trials parameter
#   is irrelevant. In this example our search space will complete in 16 trials
study.optimize(objective)

[I 2026-02-05 19:19:30,934] A new study created in memory with name: no-name-c92ce928-8995-47da-a3c6-ee89c9838dd4
[I 2026-02-05 19:19:31,436] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 19:19:32,200] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 19:19:32,497] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-05 19:19:33,025] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-05 19:19:33,562] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [17]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [18]:
# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations
- Optuna Provide magnificiant visualization graphs based on all the trials it has run and give many different beautiful insights.

In [19]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [20]:
# 1. Optimization History
plot_optimization_history(study).show()

In [21]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [22]:
# 3. Slice Plot
plot_slice(study).show()

In [23]:
# 4. Contour Plot
plot_contour(study).show()

In [24]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models
- Optuna can also be used to suggest us the best model for our data

In [25]:
# Importing the required models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [38]:
# Now Creating the Objective function according to our needs
def objective(trial):
  # Getting the suggested classifier name which will be a categorical suggestion
  classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

  if classifier_name == 'SVM':
    # Now getting the suggested hyperparameters of SVM
    c = trial.suggest_float('C', 0.1, 100, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

  elif classifier_name == 'RandomForest':
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        bootstrap=bootstrap,
        random_state=42
    )

  elif classifier_name == 'GradientBoosting':
    n_estimators = trial.suggest_int('n_estimators',50,100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

  score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
  return score

In [39]:
# Create a study and optimize the whole thing
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # No of trials are set to be 50 to reduce the workload

[I 2026-02-05 19:25:32,745] A new study created in memory with name: no-name-43d3d47e-0724-4426-8272-48ab4c288e8d
[I 2026-02-05 19:25:32,787] Trial 0 finished with value: 0.7355679702048418 and parameters: {'classifier': 'SVM', 'C': 0.7214188276800797, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7355679702048418.
[I 2026-02-05 19:25:34,026] Trial 1 finished with value: 0.7616387337057727 and parameters: {'classifier': 'RandomForest', 'n_estimators': 164, 'max_depth': 11, 'min_samples_split': 7, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 1 with value: 0.7616387337057727.
[I 2026-02-05 19:25:34,828] Trial 2 finished with value: 0.7579143389199254 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 86, 'learning_rate': 0.1440071861002202, 'max_depth': 7, 'min_samples_split': 10, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.7616387337057727.
[I 2026-02-05 19:25:35,450] Trial 3 finished with value: 0.7746741154562384 and parameter

In [40]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.6678407323740426, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7858472998137803


In [41]:
# The complete history of study is stored in a DataFrame as follows
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.735568,2026-02-05 19:25:32.747893,2026-02-05 19:25:32.787507,0 days 00:00:00.039614,0.721419,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.761639,2026-02-05 19:25:32.790676,2026-02-05 19:25:34.026185,0 days 00:00:01.235509,NaN,True,RandomForest,NaN,NaN,NaN,11.0,4.0,7.0,164.0,COMPLETE
2,2,0.757914,2026-02-05 19:25:34.027117,2026-02-05 19:25:34.828209,0 days 00:00:00.801092,NaN,NaN,GradientBoosting,NaN,NaN,0.144007,7.0,5.0,10.0,86.0,COMPLETE
3,3,0.774674,2026-02-05 19:25:34.829158,2026-02-05 19:25:35.450223,0 days 00:00:00.621065,NaN,True,RandomForest,NaN,NaN,NaN,14.0,5.0,3.0,126.0,COMPLETE
4,4,0.724395,2026-02-05 19:25:35.451889,2026-02-05 19:25:35.488520,0 days 00:00:00.036631,1.368636,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
5,5,0.718808,2026-02-05 19:25:35.489707,2026-02-05 19:25:35.555565,0 days 00:00:00.065858,86.404259,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
6,6,0.759777,2026-02-05 19:25:35.556369,2026-02-05 19:25:37.044040,0 days 00:00:01.487671,NaN,NaN,GradientBoosting,NaN,NaN,0.025983,13.0,7.0,7.0,94.0,COMPLETE
7,7,0.731844,2026-02-05 19:25:37.046393,2026-02-05 19:25:38.459361,0 days 00:00:01.412968,NaN,NaN,GradientBoosting,NaN,NaN,0.044789,19.0,2.0,5.0,58.0,COMPLETE
8,8,0.767225,2026-02-05 19:25:38.460353,2026-02-05 19:25:39.566612,0 days 00:00:01.106259,NaN,True,RandomForest,NaN,NaN,NaN,17.0,5.0,8.0,225.0,COMPLETE
9,9,0.785847,2026-02-05 19:25:39.567512,2026-02-05 19:25:39.688851,0 days 00:00:00.121339,38.812423,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [43]:
# Checking which model was most used in tuning
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,36
RandomForest,7
GradientBoosting,7


In [44]:
# Getting the Average accuracy value grouped by model
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.740090
RandomForest,0.767491
SVM,0.768363


In [45]:
# 1. Optimization History
plot_optimization_history(study).show()

In [46]:
# 3. Slice Plot
plot_slice(study).show()

In [47]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## XGBOOST HPERPARAMETER TUNING
- This section is specific for the hyperparameter tuning of XGBOOST Algorithm

In [48]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.0 MB/s eta 0:00:00


In [50]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Loading the iris dataset and directly returning the Feature Matrix and Labels
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
  # Defining the Hyperparameters along with the respective search space
  param = {
      'verbosity': 0,                   # Tells the logging type
      'objective': 'multi:softprob',    # Specifies the kind of problem we need to solve
      'num_class': 3,                   # Specifies the number of classes that can be the output
      'eval_metric': 'mlogloss',        # Specifies the evaluation criteria
      'booster': 'gbtree',              # Specifies the booster algorithm
      'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),           # Specifies the L2 Regularization search space and suggestion
      'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),             # Specifies the L1 Regularization search space and suggestion
      'eta': trial.suggest_float('eta', 0.01,0.3),                            # Specifies the learning_rate aka eta for the xgboost algo
      'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),             # Specifies the minimum loss reduction required to make a further partition on a leaf node
      'max_depth': trial.suggest_int('max_depth', 3, 9),                      # Specifies the maximum depth of the tree
      'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),       # Specifies the minimum weight for a child to carry
      'subsample': trial.suggest_float('subsample', 0.4, 1.0),                # Specifies the fraction of data to be randomly sampled for each tree
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),  # Specifies the fraction of features to be randomly sampled for each tree
      'n_estimators': 300               # Hardcoding the No_Of_Estimators means No of Trees to be only 300
  }

  # Creating the DMatrices for XGBoost
  dtrain = xgb.DMatrix(X_train, label=y_train)
  dtest = xgb.DMatrix(X_test, label=y_test)

  # Defining a pruning callback based on evaluation metrics
  # Pruning Callback is a mechanism used to stop "unpromising" trials early to
  #   save time and computational resources. Means it is essentially an Early
  #   Stopping Mechanism
  pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'eval-mlogloss')    # Evaluation criteria name means name in `eval-"name"` must be same to that of metric defined in the params

  # Training the model
  bst = xgb.train(
      param,
      dtrain,
      num_boost_round = 300,
      evals = [(dtrain, 'train'), (dtest,'eval')],    # The naming with respective data matrix is essential
      early_stopping_rounds = 30,
      callbacks = [pruning_callback]
  )

  # Now getting prediction on test set
  preds = bst.predict(dtest)
  best_preds = [int(np.argmax(line)) for line in preds]

  # Calculating the accuracy and returning for optimization
  accuracy = accuracy_score(y_test, best_preds)
  return accuracy

# Create a study with specifing a pruner algorithm
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_params}")
print(f"Best accuracy: {study.best_value}")


[I 2026-02-05 19:26:59,983] A new study created in memory with name: no-name-7859ae25-d7cf-46b0-b189-859776057237


[0]	train-mlogloss:1.04846	eval-mlogloss:1.04568
[1]	train-mlogloss:0.97660	eval-mlogloss:0.97016
[2]	train-mlogloss:0.91149	eval-mlogloss:0.90233
[3]	train-mlogloss:0.85069	eval-mlogloss:0.84049
[4]	train-mlogloss:0.79660	eval-mlogloss:0.78495
[5]	train-mlogloss:0.74561	eval-mlogloss:0.73074
[6]	train-mlogloss:0.70083	eval-mlogloss:0.68534
[7]	train-mlogloss:0.66051	eval-mlogloss:0.64218
[8]	train-mlogloss:0.62325	eval-mlogloss:0.60285
[9]	train-mlogloss:0.58742	eval-mlogloss:0.56491
[10]	train-mlogloss:0.55527	eval-mlogloss:0.53051
[11]	train-mlogloss:0.52617	eval-mlogloss:0.49871
[12]	train-mlogloss:0.50892	eval-mlogloss:0.48118
[13]	train-mlogloss:0.48223	eval-mlogloss:0.45194
[14]	train-mlogloss:0.46773	eval-mlogloss:0.43761
[15]	train-mlogloss:0.44581	eval-mlogloss:0.41406
[16]	train-mlogloss:0.42611	eval-mlogloss:0.39244
[17]	train-mlogloss:0.40955	eval-mlogloss:0.37612
[18]	train-mlogloss:0.39036	eval-mlogloss:0.35598
[19]	train-mlogloss:0.37539	eval-mlogloss:0.34040
[20]	train

[I 2026-02-05 19:27:03,061] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.0008405051442686007, 'alpha': 0.011487678754112928, 'eta': 0.06039752199695949, 'gamma': 0.010865646764670498, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.6924512054384506, 'colsample_bytree': 0.749489586388357}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.96138	eval-mlogloss:0.94932
[1]	train-mlogloss:0.84788	eval-mlogloss:0.83274
[2]	train-mlogloss:0.74986	eval-mlogloss:0.72902
[3]	train-mlogloss:0.66656	eval-mlogloss:0.64338
[4]	train-mlogloss:0.59443	eval-mlogloss:0.57428
[5]	train-mlogloss:0.53248	eval-mlogloss:0.50813
[6]	train-mlogloss:0.48056	eval-mlogloss:0.45480
[7]	train-mlogloss:0.43447	eval-mlogloss:0.41110
[8]	train-mlogloss:0.39560	eval-mlogloss:0.36840
[9]	train-mlogloss:0.36082	eval-mlogloss:0.32993
[10]	train-mlogloss:0.33055	eval-mlogloss:0.29700
[11]	train-mlogloss:0.30376	eval-mlogloss:0.26763
[12]	train-mlogloss:0.27983	eval-mlogloss:0.24430
[13]	train-mlogloss:0.25972	eval-mlogloss:0.22291
[14]	train-mlogloss:0.23931	eval-mlogloss:0.20196
[15]	train-mlogloss:0.22232	eval-mlogloss:0.18271
[16]	train-mlogloss:0.20744	eval-mlogloss:0.16611
[17]	train-mlogloss:0.19387	eval-mlogloss:0.15136
[18]	train-mlogloss:0.18251	eval-mlogloss:0.13831
[19]	train-mlogloss:0.17130	eval-mlogloss:0.12714
[20]	train

[I 2026-02-05 19:27:04,715] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.2033582359387674e-07, 'alpha': 0.0004022753407447761, 'eta': 0.1093195303632657, 'gamma': 2.430283175327988e-08, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.6196913748984023, 'colsample_bytree': 0.9061658462066308}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.87079	eval-mlogloss:0.87865
[1]	train-mlogloss:0.63493	eval-mlogloss:0.62658


[I 2026-02-05 19:27:10,238] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.88242	eval-mlogloss:0.86952
[1]	train-mlogloss:0.72393	eval-mlogloss:0.71097


[I 2026-02-05 19:27:10,277] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 0.0008405051442686007, 'alpha': 0.011487678754112928, 'eta': 0.06039752199695949, 'gamma': 0.010865646764670498, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.6924512054384506, 'colsample_bytree': 0.749489586388357}
Best accuracy: 1.0


In [51]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()